# Build and log the trip bot

A bot is here defined as an app which uses LLM. It can be an agent, 
if it takes autonomous decisions.

Logging enables us to track bot evaluations, and deploy the bot.

Run this notebook on the serverless cluster.

### Disclaimer: The current bot is a python tool bot, not a trip bot.

TODO: Hopefully it will be replaced before we run the workshop.
It should be a bot which queries trip data.

In [0]:
##Install dependencies
Ignore the warning regarding "Core Python package version(s) changed"

In [0]:
%run ../../../libs/botops/buildsetup_serverless

## Run bot

Run the bot in the context of this notebook, to see if it works, ignore the warning about "UC function client not set"

In [0]:
dbutils.library.restartPython()

In [0]:

from bot import BOT

BOT.predict({"messages": [{"role": "user", "content": "What is 4*3 in Python?"}]})


## Consume the predict stream from the bot

TODO explain

In [0]:
for chunk in BOT.predict_stream(
    {"messages": [{"role": "user", "content": "What is 4*3 in Python?"}]}
):
    print(chunk, "-----------\n")

## Log the bot as an MLflow model

Log the bot as code from the `bot.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

This enables us to track the version of the bot, and evaluations as experiments, as well as deploy it with unity catalog.

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
from bot import LLM_ENDPOINT_NAME, UC_TOOL_NAMES, VECTOR_SEARCH_TOOLS
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint

# Pass the llm endpoint as a resource to enable automatic authentication
resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in VECTOR_SEARCH_TOOLS:
    resources.extend(tool.resources)
for tool_name in UC_TOOL_NAMES:
    resources.append(DatabricksFunction(function_name=tool_name))

with mlflow.start_run():  # A run enable mlflow registration, logging and tracking
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="tripbot",
        python_model="bot.py",
        pip_requirements=[
            "mlflow",
            "backoff",
            "databricks-openai",
        ],
        resources=resources,
    )

## Show the details of the logging

In [0]:
logged_agent_info.run_id

In [0]:
logged_agent_info.model_uri

In [0]:
logged_agent_info.artifact_path

In [0]:
logged_agent_info.metadata

Now let us evaluate our model before we deploy it, in the next notebook.